In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision import transforms, models, datasets
import torchvision
import numpy as np
import h5py
from matplotlib import pyplot as plt
from Utils import save_large_dataset, load_large_dataset, calculate_metric
plt.ion()   # interactive mode

In [2]:
X = load_large_dataset('images')
Y = load_large_dataset('labels')

In [3]:
X = X.squeeze() #remove unnecessary dimension
Y = Y.squeeze()
print (X.shape)

(1792, 121, 145, 121)


In [4]:
list_classifiers = ["1_36_39", "2_31_34", "1_60_63", "1_75_78", "2_46_49",
                    "2_55_58", "3_13_16", "3_19_22", "3_31_34", "3_55_58", "2_40_43"]

In [5]:
net = models.vgg11_bn()
net.classifier._modules['6'] = nn.Linear(4096, 2)
net.cuda()
net.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (6): ReLU(inplace)
    (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (10): ReLU(inplace)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (13): ReLU(inplace)
    (14): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), st

In [6]:
np.random.seed(0) #seed is the same as used in the training phase
mask = np.random.rand(len(X)) < 0.9  #array of boolean variables
validation_set = X[~mask]
validation_labels = Y[~mask]

In [7]:
print ('Reported metric is (sensitivity + specificity)/2\n')
aggregated_preds = np.zeros(len(validation_set))

for i in range(len(list_classifiers)):

    axis = int(list_classifiers[i].split("_")[0])
    start_index = int(list_classifiers[i].split("_")[1])
    end_index = int(list_classifiers[i].split("_")[2])
    
    if (axis==1):
        X_sliced = X[:,start_index:end_index,:,:] #take only 3 slices and treat them as channels
        X_padded = np.pad(X_sliced,((0,0),(0,0),(40,39),(51,52)), 'constant') #pad with zeros to get 224x224
        X_padded = np.float32(X_padded)
        X_padded = np.rollaxis(X_padded, axis, 1) #move channel dimension to be the first one
    
    if (axis==2):
        X_sliced = X[:,:,start_index:end_index,:] #take only 3 slices and treat them as channels
        X_padded = np.pad(X_sliced,((0,0),(51,52),(0,0),(51,52)), 'constant') #pad with zeros to get 224x224
        X_padded = np.float32(X_padded)
        X_padded = np.rollaxis(X_padded, axis, 1) #move channel dimension to be the first one
        
    if (axis==3):
        X_sliced = X[:,:,:,start_index:end_index] #take only 3 slices and treat them as channels
        X_padded = np.pad(X_sliced,((0,0),(51,52),(40,39),(0,0)), 'constant') #pad with zeros to get 224x224
        X_padded = np.float32(X_padded)
        X_padded = np.rollaxis(X_padded, axis, 1) #move channel dimension to be the first one
        
    np.random.seed(0) #seed is the same as used in the training phase
    mask = np.random.rand(len(X)) < 0.9  #array of boolean variables
    validation_set = X_padded[~mask]
    validation_labels = Y[~mask]
    validation_set = torch.from_numpy(validation_set) #convert to torch tensor

    net.load_state_dict(torch.load("./trained_classifiers/" + list_classifiers[i] + ".pt")) #loading weights
    
    preds = np.zeros((len(validation_set)))
    for z in range(len(validation_set)):
        output = net(Variable(validation_set[z].unsqueeze(0).cuda()))
        _, preds[z] = torch.max(output.data.cpu(), 1)
        aggregated_preds[z] += preds[z]
    
    print ('Classifier ' + list_classifiers[i] + ': {:.4f}'.format(calculate_metric(preds,validation_labels)))

ensemble_preds = (aggregated_preds/len(list_classifiers))
ensemble_preds = np.rint(ensemble_preds) #round to closest integer (0 or 1)
print ("----------")
print ('Metric of Ensemble Model {:.4f}'.format(calculate_metric(ensemble_preds,validation_labels)))
accuracy = (np.count_nonzero(ensemble_preds==validation_labels))/len(validation_labels)
print ('Accuracy of Ensemble Model: {:.4f}'.format(accuracy))

Reported metric is (sensitivity + specificity)/2

Classifier 1_36_39: 0.6511
Classifier 2_31_34: 0.6295
Classifier 1_60_63: 0.6223
Classifier 1_75_78: 0.6319
Classifier 2_46_49: 0.6353
Classifier 2_55_58: 0.6168
Classifier 3_13_16: 0.6195
Classifier 3_19_22: 0.6356
Classifier 3_31_34: 0.6429
Classifier 3_55_58: 0.6765
Classifier 2_40_43: 0.6676
----------
Metric of Ensemble Model 0.6961
Accuracy of Ensemble Model: 0.7044
